In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
lookup = 'Forsa Hartmoor'
motor_num = 0

In [ ]:
def sfun(x):
    #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')

In [ ]:
fleet.T

In [ ]:
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
#success = [True,False]
success = [True]
e=Engine.from_fleet(mp,motor)
pp_from=e['Commissioning Date']
#pp_from='2022-03-28 06:00'
#pp_to='2022-03-28 08:14'
pp_to=datetime.now()
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
# Run State Machines
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = False)

In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
#rda[filterFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
from ipywidgets import interact_manual

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

In [ ]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

In [ ]:
p = figure(title="simple line example", height=300, width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [ ]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    print('i am called')
    push_notebook(handle=fig_handle)

In [ ]:
vv = 0
def update_fig(x=0):
    global vv; vv = x; startversuch = rdb.iloc[x]
    #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
    #display(HTML(ftitle));
    disp_result(startversuch)
    for h in fig_handles:
        push_notebook(handle=h)

In [ ]:
# PLotter
def myfigures(e):
    return {
    'actors' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
    ],
    'lubrication' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black', 'unit': 'bar'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
    {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
    ],
    'exhaust' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
    {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 4000], 'unit':'V'},
    ],
    'ignition' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
    {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
    {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
    ],
        
}
lfigures = myfigures(fsm._e)
plotdef, vset = cplotdef(mp, lfigures)
#dmaxlength = 1800
dmaxlength = None
dminlength = None
startversuch = rdb.iloc[vv]
ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
fig_handles = []
for doplot in plotdef:
    dset = lfigures[doplot]
    ltitle = f"{ftitle} | {doplot}"
    if count_columns(dset) > 12: # no legend, if too many lines.
        fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=(18, 10))
    else:
        fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=(18, 10))

    fig = FSM_add_Notations(fig, fsm, startversuch)
    disp_alarms(startversuch)
    disp_warnings(startversuch)
    fig = FSM_add_Alarms(fig, fsm, startversuch)
    fig = FSM_add_Warnings(fig, fsm, startversuch)
    fig_handles.append(bokeh_show(fig, notebook_handle=True))

In [ ]:
#interact_manual(update_fig, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='90%')));
interact(update_fig, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='90%')));

In [ ]:
fig_handle = show(p, notebook_handle=True)

In [ ]:
interact_manual(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))